# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

<div class="span5 alert alert-info">
### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet


#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
</div>
****

In [1]:
import pandas as pd
import numpy as np
from scipy import stats

In [2]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [3]:
# number of callbacks for black-sounding names
sum(data[data.race=='b'].call)

157.0

In [6]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


### 1. 

For this problem, I will use a two-sample T-test for difference between means. I will likely choose a two-tailed test because we are testing if any difference exists.

In [7]:
len(data)

4870

Since n = 4870, the CLT applies


### 2. 

The null hypothesis is that there is no difference in the proportion of resumes given call backs between black-sounding names and white-sounding names.

The alternative hypothesis is that there is a difference in the proportion of resumes given call backs.

### 3.

In [10]:
bNum = len(data[data.race == 'b'])
bProportion = sum(data[data.race=='b'].call)/bNum
bProportion, bNum

(0.064476386036960986, 2435)

In [11]:
wNum = len(data[data.race == 'w'])
wProportion = sum(data[data.race=='w'].call)/bNum
wProportion, wNum

(0.096509240246406572, 2435)

In [26]:
diffMean = wProportion - bProportion
diffSD = np.sqrt(data[data.race=='b']['call'].std()**2/bNum + data[data.race=='w']['call'].std()**2/wNum)
diffMean, diffSD


(0.032032854209445585, 0.0077849069198137931)

For an alpha of .05, we can create a two-tailed 95% confidence interval by finding a range +/- 1.96 standard deviations from the mean

In [27]:
(diffMean - 1.96*diffSD, diffMean + 1.96*diffSD)

(0.016774436646610554, 0.047291271772280617)

We are 95% confident that the true difference between black-sounding and white-sounding names is captured in the above range.

In [29]:
popProportion = sum(data.call)/(bNum+wNum)
popProportion

0.080492813141683772

In [31]:
nullSD = np.sqrt(2*data['call'].std()**2/(bNum+wNum))
nullSD

0.0055137394402213187

In [38]:
z = diffMean / nullSD
p_value = 1-(stats.norm.cdf(z) - stats.norm.cdf(-z))
p_value

6.2606437811751903e-09

For an alpha of .05, we reject the null hypothesis that there is no difference in the proportion of resumes given callbacks between black-sounding and white-sounding names. Our p-value is 6.26 x 10^-9.

### 4. 

The data strongly suggests that race has a significant impact on the rate of callbacks for resumes. We are 95% confident that the difference in rate of callbacks between white-sounding names and black-sounding names is captured in the interval (.01677, .04729). Black-sounding names receive call backs at a rate 17% to 49% lower than the call-back rate for white-sounding names. We reject the null hypothesis that there is no difference in the proportion of callbacks with a p = 6.26 x 10^-9.

### 5.

Race is undoubtedly a very significant factor, but our analysis has not shown it to be the most important factor. We have not explored the effect of other variables on the call-back rate. To determine whether race is the most important factor, we would need to see the difference in call-back rates segmenting for other attributes.